In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import xgboost as xgb
from sklearn import preprocessing
from keras.models import Sequential
from keras.layers import Dense, LSTM
from keras.layers.embeddings import Embedding
df1 = pd.read_csv("data/train_preliminary/user.csv")
df2 = pd.read_csv("data/train_preliminary/ad.csv")
df3 = pd.read_csv("data/train_preliminary/click_log.csv")
df_tmp = pd.merge(df3, df1, left_on='user_id', right_on='user_id')
df = pd.merge(df_tmp, df2, left_on='creative_id', right_on='creative_id')
order = ['user_id', 'time', 'creative_id', 'click_times', 'ad_id', 'product_category', 'advertiser_id']
df = df[order]
df.head()

Using TensorFlow backend.


,user_id,time,creative_id,click_times,ad_id,product_category,advertiser_id
0,30920,9,567330,1,504423,3,32638
1,320815,15,567330,1,504423,3,32638
2,355089,11,567330,1,504423,3,32638
3,363442,9,567330,1,504423,3,32638
4,370513,14,567330,1,504423,3,32638


In [2]:
df.sort_values(['user_id', 'time'], inplace=True)
df

,user_id,time,creative_id,click_times,ad_id,product_category,advertiser_id
16204330,1,20,877468,1,773445,5,29455
23067470,1,20,209778,1,188507,2,9702
26781033,1,20,821396,1,724607,5,7293
22502290,1,39,1683713,1,1458878,5,14668
9504689,1,40,122032,1,109959,2,11411
...,...,...,...,...,...,...,...
28703035,900000,57,2692144,1,2316309,5,14372
15724163,900000,71,3391205,1,2918341,5,22137
7226547,900000,72,3471208,1,2989104,5,7283
9638586,900000,74,3527098,1,3037879,18,14681


In [3]:
df.reset_index(drop=True, inplace=True)
df

,user_id,time,creative_id,click_times,ad_id,product_category,advertiser_id
0,1,20,877468,1,773445,5,29455
1,1,20,209778,1,188507,2,9702
2,1,20,821396,1,724607,5,7293
3,1,39,1683713,1,1458878,5,14668
4,1,40,122032,1,109959,2,11411
...,...,...,...,...,...,...,...
30082766,900000,57,2692144,1,2316309,5,14372
30082767,900000,71,3391205,1,2918341,5,22137
30082768,900000,72,3471208,1,2989104,5,7283
30082769,900000,74,3527098,1,3037879,18,14681


In [4]:
df['creative_id'].max()

4445718

In [5]:
# creative_id embedding后 作为特征
x_tmp = np.array(df['creative_id'])
x_tmp = x_tmp.reshape((-1, 1))
x_tmp = np.vstack((x_tmp, np.zeros((29, 1))))
print(x_tmp.shape)
x_tmp = x_tmp.reshape((300828, 100))
model = Sequential()
model.add(Embedding(4445719, 5, input_length=100))
model.compile(loss='MSE', optimizer='adam')
y_tmp = model.predict(x_tmp, verbose=1)
y_tmp = y_tmp.reshape((30082800, 5))
X_tmp = pd.DataFrame(y_tmp[:30082771, :], columns=['creative_id_0', 'creative_id_1', 'creative_id_2', 'creative_id_3', 'creative_id_4'])
X_tmp

(30082800, 1)
300828/300828 [==============================] - 3s 11us/step


,creative_id_0,creative_id_1,creative_id_2,creative_id_3,creative_id_4
0,0.016948,-0.021450,0.033767,0.026174,-0.000460
1,0.029825,-0.023604,-0.013073,-0.019842,-0.022133
2,-0.009569,0.004028,-0.032061,0.035826,-0.006448
3,0.017495,0.045433,0.036214,0.029466,-0.012365
4,-0.017028,-0.033311,0.039625,-0.010888,0.029749
...,...,...,...,...,...
30082766,-0.013067,0.006342,-0.040982,0.033658,0.036188
30082767,-0.049668,0.036928,-0.041257,-0.041968,-0.030981
30082768,0.023935,0.014689,-0.022151,-0.007241,0.027704
30082769,0.018831,0.011576,0.030707,0.025318,-0.022741


In [6]:
# time click_times 直接作为特征
X = df[['user_id', 'time', 'click_times']]

In [7]:
X = pd.concat([X, X_tmp], axis=1)
X.head()

,user_id,time,click_times,creative_id_0,creative_id_1,creative_id_2,creative_id_3,creative_id_4
0,1,20,1,0.016948,-0.021450,0.033767,0.026174,-0.000460
1,1,20,1,0.029825,-0.023604,-0.013073,-0.019842,-0.022133
2,1,20,1,-0.009569,0.004028,-0.032061,0.035826,-0.006448
3,1,39,1,0.017495,0.045433,0.036214,0.029466,-0.012365
4,1,40,1,-0.017028,-0.033311,0.039625,-0.010888,0.029749


In [8]:
df['ad_id'].max()

3812200

In [9]:
# ad_id embedding后 作为特征
x_tmp = np.array(df['ad_id'])
x_tmp = x_tmp.reshape((-1, 1))
x_tmp = np.vstack((x_tmp, np.zeros((29, 1))))
print(x_tmp.shape)
x_tmp = x_tmp.reshape((300828, 100))
model = Sequential()
model.add(Embedding(3812201, 5, input_length=100))
model.compile(loss='MSE', optimizer='adam')
y_tmp = model.predict(x_tmp, verbose=1)
y_tmp = y_tmp.reshape((30082800, 5))
X_tmp = pd.DataFrame(y_tmp[:30082771, :], columns=['ad_id_0', 'ad_id_1', 'ad_id_2', 'ad_id_3', 'ad_id_4'])
X_tmp

(30082800, 1)
300828/300828 [==============================] - 4s 12us/step


,ad_id_0,ad_id_1,ad_id_2,ad_id_3,ad_id_4
0,0.025098,-0.038430,-0.041154,-0.025835,0.024350
1,0.035635,-0.039314,0.011658,0.049518,-0.026567
2,-0.012626,0.017264,-0.044100,0.049653,0.011014
3,-0.041934,0.033759,0.013967,0.022001,0.006169
4,0.015961,0.030117,0.007503,0.008061,0.004176
...,...,...,...,...,...
30082766,0.016650,-0.016962,0.038166,0.049928,-0.010943
30082767,0.016385,-0.012537,0.000259,-0.030670,-0.043168
30082768,-0.015498,-0.017219,-0.018528,0.009995,-0.011202
30082769,-0.044445,-0.043361,-0.036139,-0.018770,-0.004615


In [10]:
X = pd.concat([X, X_tmp], axis=1)
X.head()

,user_id,time,click_times,creative_id_0,creative_id_1,creative_id_2,creative_id_3,creative_id_4,ad_id_0,ad_id_1,ad_id_2,ad_id_3,ad_id_4
0,1,20,1,0.016948,-0.021450,0.033767,0.026174,-0.000460,0.025098,-0.038430,-0.041154,-0.025835,0.024350
1,1,20,1,0.029825,-0.023604,-0.013073,-0.019842,-0.022133,0.035635,-0.039314,0.011658,0.049518,-0.026567
2,1,20,1,-0.009569,0.004028,-0.032061,0.035826,-0.006448,-0.012626,0.017264,-0.044100,0.049653,0.011014
3,1,39,1,0.017495,0.045433,0.036214,0.029466,-0.012365,-0.041934,0.033759,0.013967,0.022001,0.006169
4,1,40,1,-0.017028,-0.033311,0.039625,-0.010888,0.029749,0.015961,0.030117,0.007503,0.008061,0.004176


In [11]:
df['product_category'].max()

18

In [12]:
# product_category embedding后 作为特征
x_tmp = np.array(df['product_category'])
x_tmp = x_tmp.reshape((-1, 1))
x_tmp = np.vstack((x_tmp, np.zeros((29, 1))))
print(x_tmp.shape)
x_tmp = x_tmp.reshape((300828, 100))
model = Sequential()
model.add(Embedding(20, 5, input_length=100))
model.compile(loss='MSE', optimizer='adam')
y_tmp = model.predict(x_tmp, verbose=1)
y_tmp = y_tmp.reshape((30082800, 5))
X_tmp = pd.DataFrame(y_tmp[:30082771, :], columns=['product_category_0', 'product_category_1', 
                                                   'product_category_2', 'product_category_3', 'product_category_4'])
X = pd.concat([X, X_tmp], axis=1)
X.head()

(30082800, 1)
300828/300828 [==============================] - 3s 10us/step


,user_id,time,click_times,creative_id_0,creative_id_1,creative_id_2,creative_id_3,creative_id_4,ad_id_0,ad_id_1,ad_id_2,ad_id_3,ad_id_4,product_category_0,product_category_1,product_category_2,product_category_3,product_category_4
0,1,20,1,0.016948,-0.021450,0.033767,0.026174,-0.000460,0.025098,-0.038430,-0.041154,-0.025835,0.024350,0.022657,0.004967,-0.040322,-0.020464,0.041494
1,1,20,1,0.029825,-0.023604,-0.013073,-0.019842,-0.022133,0.035635,-0.039314,0.011658,0.049518,-0.026567,-0.013614,0.005858,0.023829,-0.026948,0.032221
2,1,20,1,-0.009569,0.004028,-0.032061,0.035826,-0.006448,-0.012626,0.017264,-0.044100,0.049653,0.011014,0.022657,0.004967,-0.040322,-0.020464,0.041494
3,1,39,1,0.017495,0.045433,0.036214,0.029466,-0.012365,-0.041934,0.033759,0.013967,0.022001,0.006169,0.022657,0.004967,-0.040322,-0.020464,0.041494
4,1,40,1,-0.017028,-0.033311,0.039625,-0.010888,0.029749,0.015961,0.030117,0.007503,0.008061,0.004176,-0.013614,0.005858,0.023829,-0.026948,0.032221


In [13]:
df['advertiser_id'].max()

62965

In [14]:
# advertiser_id embedding后 作为特征
x_tmp = np.array(df['advertiser_id'])
x_tmp = x_tmp.reshape((-1, 1))
x_tmp = np.vstack((x_tmp, np.zeros((29, 1))))
x_tmp = x_tmp.reshape((300828, 100))
model = Sequential()
model.add(Embedding(62966, 5, input_length=100))
model.compile(loss='MSE', optimizer='adam')
y_tmp = model.predict(x_tmp, verbose=1)
y_tmp = y_tmp.reshape((30082800, 5))
X_tmp = pd.DataFrame(y_tmp[:30082771, :], columns=['advertiser_id_0', 'advertiser_id_1', 
                                                   'advertiser_id_2', 'advertiser_id_3', 'advertiser_id_4'])
X = pd.concat([X, X_tmp], axis=1)
X.head()

300828/300828 [==============================] - 2s 8us/step


,user_id,time,click_times,creative_id_0,creative_id_1,creative_id_2,creative_id_3,creative_id_4,ad_id_0,ad_id_1,...,product_category_0,product_category_1,product_category_2,product_category_3,product_category_4,advertiser_id_0,advertiser_id_1,advertiser_id_2,advertiser_id_3,advertiser_id_4
0,1,20,1,0.016948,-0.021450,0.033767,0.026174,-0.000460,0.025098,-0.038430,...,0.022657,0.004967,-0.040322,-0.020464,0.041494,-0.030044,-0.032493,-0.002481,0.022452,0.008597
1,1,20,1,0.029825,-0.023604,-0.013073,-0.019842,-0.022133,0.035635,-0.039314,...,-0.013614,0.005858,0.023829,-0.026948,0.032221,-0.042298,-0.040606,0.031544,-0.015485,-0.049409
2,1,20,1,-0.009569,0.004028,-0.032061,0.035826,-0.006448,-0.012626,0.017264,...,0.022657,0.004967,-0.040322,-0.020464,0.041494,0.026815,0.014794,0.020790,-0.009151,0.030901
3,1,39,1,0.017495,0.045433,0.036214,0.029466,-0.012365,-0.041934,0.033759,...,0.022657,0.004967,-0.040322,-0.020464,0.041494,-0.010593,-0.000313,-0.023450,-0.039842,0.018362
4,1,40,1,-0.017028,-0.033311,0.039625,-0.010888,0.029749,0.015961,0.030117,...,-0.013614,0.005858,0.023829,-0.026948,0.032221,-0.009999,-0.007316,0.044682,-0.044602,0.015583


In [15]:
# 对齐 为lstm作准备 对齐成(-1, 30, 22)
def check_matrix(x):
    if x.shape[0] >= 30:
        return x.iloc[:30, :]
    else:
        tmp1 = np.zeros((30 - x.shape[0], x.shape[1]))
        tmp1 = pd.DataFrame(tmp1)
        tmp1.columns = x.columns
        return pd.concat([x, tmp1], axis=0)
X = X.groupby('user_id').apply(check_matrix)
X = X.reset_index(drop=True)
X = X.drop('user_id', axis = 1)
X

,time,click_times,creative_id_0,creative_id_1,creative_id_2,creative_id_3,creative_id_4,ad_id_0,ad_id_1,ad_id_2,...,product_category_0,product_category_1,product_category_2,product_category_3,product_category_4,advertiser_id_0,advertiser_id_1,advertiser_id_2,advertiser_id_3,advertiser_id_4
0,20.0,1.0,0.016948,-0.021450,0.033767,0.026174,-0.000460,0.025098,-0.038430,-0.041154,...,0.022657,0.004967,-0.040322,-0.020464,0.041494,-0.030044,-0.032493,-0.002481,0.022452,0.008597
1,20.0,1.0,0.029825,-0.023604,-0.013073,-0.019842,-0.022133,0.035635,-0.039314,0.011658,...,-0.013614,0.005858,0.023829,-0.026948,0.032221,-0.042298,-0.040606,0.031544,-0.015485,-0.049409
2,20.0,1.0,-0.009569,0.004028,-0.032061,0.035826,-0.006448,-0.012626,0.017264,-0.044100,...,0.022657,0.004967,-0.040322,-0.020464,0.041494,0.026815,0.014794,0.020790,-0.009151,0.030901
3,39.0,1.0,0.017495,0.045433,0.036214,0.029466,-0.012365,-0.041934,0.033759,0.013967,...,0.022657,0.004967,-0.040322,-0.020464,0.041494,-0.010593,-0.000313,-0.023450,-0.039842,0.018362
4,40.0,1.0,-0.017028,-0.033311,0.039625,-0.010888,0.029749,0.015961,0.030117,0.007503,...,-0.013614,0.005858,0.023829,-0.026948,0.032221,-0.009999,-0.007316,0.044682,-0.044602,0.015583
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26999995,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
26999996,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
26999997,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
26999998,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [16]:
# X.to_csv('data/train_preliminary/lstm_5.csv')

In [17]:
# 归一
X = preprocessing.MinMaxScaler().fit_transform(X)


In [18]:
X = np.array(X).reshape((900000, 30, 22))

In [19]:
# 预测性别
from keras.layers.core import Masking
X_train, X_test = X[:800000, :, :], X[800000:, :, :]
Y2_train, Y2_test = df1.iloc[:800000, 2:3], df1.iloc[800000:, 2:3]
model = Sequential()
model.add(Masking(mask_value=0., input_shape=(30, 22)))
model.add(LSTM(32))
model.add(Dense(1))
model.compile(loss='MSE', optimizer='adam')
model.fit(X_train, Y2_train, nb_epoch=2)

/Users/lixiang/Documents/GitHub/xzwbsrz_pre/venv/lib/python3.6/site-packages/ipykernel_launcher.py:10: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  # Remove the CWD from sys.path while we load stuff.


Epoch 1/2
800000/800000 [==============================] - 521s 651us/step - loss: 0.2038
Epoch 2/2
800000/800000 [==============================] - 512s 641us/step - loss: 0.1815


In [20]:
Y2_train.shape

(800000, 1)

In [21]:
Y = model.predict(X_train)
Y = np.round(Y)
ans = 0
for i in range(800000):
    if Y[i] == Y2_train.iloc[i, 0]:
        ans += 1
print('accurency:', ans/800000)

accurency: 0.74097375


In [22]:
Y = model.predict(X_test)
Y = np.round(Y)
ans = 0
for i in range(100000):
    if Y[i] == Y2_test.iloc[i, 0]:
        ans += 1
print('accurency:', ans/100000)

accurency: 0.74146


In [26]:
X_ready = model.predict(X)
X_ready

array([[1.3275533],
       [1.3892401],
       [1.8179817],
       ...,
       [1.9805512],
       [1.3113501],
       [1.4810032]], dtype=float32)

In [27]:
X_tmp = df3.groupby('user_id')['click_times'].sum()
X_tmp

user_id
1         14
2         46
3         30
4         29
5         34
          ..
899996    14
899997    20
899998    15
899999    22
900000    12
Name: click_times, Length: 900000, dtype: int64

In [29]:
X_tmp.reset_index(drop=True, inplace=True)
X_tmp

0         14
1         46
2         30
3         29
4         34
          ..
899995    14
899996    20
899997    15
899998    22
899999    12
Name: click_times, Length: 900000, dtype: int64

In [30]:
X_ready = pd.DataFrame(X_ready, columns=['lstm'])
X_ready = pd.concat([X_ready, X_tmp], axis = 1)
X_ready

,lstm,click_times
0,1.327553,14
1,1.389240,46
2,1.817982,30
3,1.292678,29
4,1.373059,34
...,...,...
899995,1.255034,14
899996,1.533280,20
899997,1.980551,15
899998,1.311350,22


In [32]:
X_tmp1 = df3.groupby('user_id')['click_times'].mean()
X_tmp2 = df3.groupby('user_id')['click_times'].std()
X_tmp1.reset_index(drop=True, inplace=True)
X_tmp2.reset_index(drop=True, inplace=True)
X_ready = pd.concat([X_ready, X_tmp1, X_tmp2], axis = 1)

In [33]:
X_tmp = df[['user_id', 'time']].groupby('user_id')['time'].std().reset_index(drop=True)
X_tmp

0         19.585775
1         21.653562
2         26.386953
3         17.421039
4         23.186783
            ...    
899995    23.559138
899996    28.867344
899997    18.683012
899998    24.854773
899999    29.486001
Name: time, Length: 900000, dtype: float64

In [34]:
X_ready = pd.concat([X_ready, X_tmp], axis = 1)

In [35]:
X_ready = np.array(X_ready)
X_ready = preprocessing.MinMaxScaler().fit_transform(X_ready)
X_train, X_test = X_ready[:800000, :], X_ready[800000:900000, :]
model1 = xgb.XGBClassifier()
model1.fit(X_train, Y2_train)
model1.score(X_train, Y2_train)

/Users/lixiang/Documents/GitHub/xzwbsrz_pre/venv/lib/python3.6/site-packages/sklearn/preprocessing/_label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/lixiang/Documents/GitHub/xzwbsrz_pre/venv/lib/python3.6/site-packages/sklearn/preprocessing/_label.py:268: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.7438325

In [36]:
model1.score(X_test, Y2_test)

0.74161